## Indentation
- *indentation* means shifting a line of code by either a given number of spaces or a tab (`Tab` key);
- a tab is a *single* special character that is visualised as an empty space;
- tab-style indentation may have been popular in the past, but today the standard is space-style indentation using 4 whitespaces;
- most editors will produce 4 whitespaces by default (or can be set up to do so!)

### In python
- indentation in python is ***part of the syntax!***
- indentation delimits the code of a function, an `if/elif/else` clause, a loop etc.
- any number of spaces is recognised, but it has to be consistent



## Dictionaries tips and tricks

In [15]:
a_dict = { letter: number for number, letter in enumerate('abcdef') }

print(a_dict)


{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5}


In [12]:
# print(a_dict['h']) # not nice

In [18]:
print(a_dict.get('h'))

None


In [25]:
print(a_dict.get('f',-1))
print(a_dict.get('g',-1))

5
-1


## `defaultdict`
`defaultdict` is a dictionary associated to a function, the output of this function is the default value for all non-existent keys

In [50]:
from collections import defaultdict

In [58]:
def init_value():
    return "default value"

a = defaultdict(init_value)

print(a["0"])

default value


More useful when we do not know all our keys in advance.

In [59]:
a = defaultdict(list)

names = ["Xavier", "Massimiliano", "Anastasiia"]

for name in names:
    a[name].append("python")
 

print(a)

defaultdict(<class 'list'>, {'Xavier': ['python'], 'Massimiliano': ['python'], 'Anastasiia': ['python']})


# Functions

What are functions?
- from maths: relation between two sets A and B that assign to each element of A exactly one element of B
- in computing: similar concept, but in general (1) A and B are loosely defined (2) the association is determined through an algorithm.

## Example function

In [1]:
def f(a, b):
    c = a + b
    return c

In [2]:
f(3,2)

5

- `a` and `b` are called *arguments* and represent the input of the function;
- `c` is called *return value* and represent the output of the function;
- the combination of arguments and return values takes the name of *signature* of a function (not very important in `python`, much more in compiled languages);
- functions return **one** item, don't they?

In [3]:
def g(a, b):
    c = a + b
    d = a - b
    return c, d

def h():
    return
    # pass

In [4]:
sum, diff = g(3,2)
print(sum, diff)

5 1


In [5]:
print(g(3,2), type(g(3,2)))

(5, 1) <class 'tuple'>


In [6]:
print(h(), type(h()))

None <class 'NoneType'>


Functions in `python` return always exactly one item, but this item can actually be a collection. If you try to return more than one value, python will create a tuple for you.

## Positional arguments and keyword arguments
- when calling a function with a sequence of arguments, we say we are *passing* the arguments to the function;
- `python` function accepts arguments passing either by position or by keyword;
- keyword arguments must alwasy follow positional arguments!

In [30]:
def f(a, b, c):
    print(a, b, c)

f(1,2,3) # position
f(a=1, b=2, c=3) # keywords
f(1, 2, c=3) # mixed
# f(a=1, 2, 3)

1 2 3
1 2 3
1 2 3


### Do all arguments need to be defined a priori?
**No**!

In [47]:
def f(arg, *args, **kwargs):
    print(arg) # a name
    print(args) # a tuple
    print(kwargs) # a dictionary

In [48]:
f(1,2,3,d=4, e=5)

1
(2, 3)
{'d': 4, 'e': 5}


This gives you a lot of flexibility but be careful in taking advantage of it. It is not a good idea to have too little control on what is passed to your function.